# Mikhail Gulyi

# My connection

### Отправка запросов на сервер

In [4]:
import pyodbc
conn = pyodbc.connect(
    "Driver={SQL Server};"
    "Server=MIKHAILGULYIPC\SQLEXPRESS;"
    "Database=computer;"
    "Trusted_Connection=yes;"
)

In [5]:
cursor = conn.cursor()
cursor.execute('select * from Laptop')
for row in cursor:
    print(row)
cursor.close()      

(1, '1298', 350, 32, 4.0, Decimal('700.0000'), 11)
(2, '1321', 500, 64, 8.0, Decimal('970.0000'), 12)
(3, '1750', 750, 128, 12.0, Decimal('1200.0000'), 14)
(4, '1298', 600, 64, 10.0, Decimal('1050.0000'), 15)
(5, '1752', 750, 128, 10.0, Decimal('1150.0000'), 14)
(6, '1298', 450, 64, 10.0, Decimal('950.0000'), 12)


### Загрузка базы данных в датафрейм

### Использование pd.read_sql_query

In [9]:
import pandas as pd
cursor = conn.cursor()
sql_query = pd.read_sql_query('''SELECT * FROM Laptop''', con=conn)
df = pd.DataFrame(sql_query, columns=[
                  'code', 'model', 'speed', 'ram', 'hd', 'price', 'screen'])
cursor.close()                  


In [11]:
df.head()

,code,model,speed,ram,hd,price,screen
0,1,1298,350,32,4.0,700.0,11
1,2,1321,500,64,8.0,970.0,12
2,3,1750,750,128,12.0,1200.0,14
3,4,1298,600,64,10.0,1050.0,15
4,5,1752,750,128,10.0,1150.0,14


### Использование pd.read_sql

In [13]:
df = pd.read_sql('SELECT * FROM Laptop', conn)

In [14]:
df.head()

,code,model,speed,ram,hd,price,screen
0,1,1298,350,32,4.0,700.0,11
1,2,1321,500,64,8.0,970.0,12
2,3,1750,750,128,12.0,1200.0,14
3,4,1298,600,64,10.0,1050.0,15
4,5,1752,750,128,10.0,1150.0,14


In [20]:
pyodbc.drivers()

['SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)',
 'SQL Server Native Client 11.0',
 'SQL Server Native Client RDA 11.0',
 'ODBC Driver 17 for SQL Server']

### Загрузка датафрейма в базу данных

In [47]:
df = pd.read_csv('data/Breast Cancer Wisconsin (Diagnostic) Data Set.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [49]:
import urllib
import sqlalchemy as sa

connection_string = (
    "Driver={SQL Server};"
    "Server=MIKHAILGULYIPC\SQLEXPRESS;"
    "Database=Test;"
    "Trusted_Connection=yes;"
)
connection_uri = f"mssql+pyodbc:///?odbc_connect={urllib.parse.quote_plus(connection_string)}"
engine = sa.create_engine(connection_uri, fast_executemany=True)

df.to_sql("TableName", engine,
          if_exists="replace", index=False, chunksize=5)
